In [4]:
#!/usr/bin/env python3.10


from bs4 import BeautifulSoup
import pandas as pd
import re
from functions import transliterate_xml
from timeis import timeis, yellow, line, tic, toc
from clean_text import text_cleaner

# print(f"{timeis()} {line}")
# print(f"{timeis()} {yellow}building database of vinaya")
# print(f"{timeis()} {line}")

cst_dir = "../Cst4/Xml/"
output_dir = "output/"
# log_file = "log/log.tsv"
# log = open(log_file, "a")

file_list = [
    "vin01m.mul.xml",
    # "vin02m1.mul.xml",
    # "vin02m2.mul.xml",
    # "vin02m3.mul.xml",
    # "vin02m4.mul.xml"
]

for file_name in file_list:
	print(f"{timeis()} {file_name}")

	# transliterate the xml files into roman and save
	with open(f"{cst_dir}{file_name}", "r", encoding="UTF-16") as f:
		xml = f.read()

	xml = transliterate_xml(xml)

	with open(f"../Cst4/xml roman/{file_name}", "w") as w:
		w.write(xml)

	# make the soup
	soup = BeautifulSoup(xml, "xml")

	# remove all the "pb" tags
	pbs = soup.find_all("pb")
	for pb in pbs:
		pb.decompose()

	# remove all the notes
	notes = soup.find_all("note")
	for note in notes:
		note.decompose()

	# remove all the para tags
	pns = soup.find_all("p")
	for pn in pns:
		del pn["n"]

	# remove all the hi tags
	his = soup.find_all("hi")
	for hi in his:
		hi.unwrap()
	
	with open(f"output/temp/{file_name}.xml", "w") as w:
		w.write(soup.prettify())

		
	# - text cleaner
	# - find chatper title subhead
	# - list of all possible sentence beginning and endings
	# - find all inflections

search_word = input("what are you looking for? ")
search_sentence = re.compile(
	f"(^|\\. |\\! |\\? )(.*?{search_word}.*?($|\\. |\\! |\\? ))")


nikaya = soup.find_all("p", rend="nikaya")[0].string
book = soup.find_all("head", rend="book")[0].string
if book == "pārājikapāḷi":
	book_no = "1."

# kanda = div.head rend = "chapter"
# sutta = p rend = "title"
# subtitle = p rend = "subhead"

result_counter = 1
sutta_counter = 1
sutta_counter = 1
subtitle_count = 1

kanda = ""
sutta = ""
subtitle = ""

divs = soup.find_all("div", type="kanda")

for div in divs:

	# get kanda name and number
	if div.head["rend"] == "chapter":
		kanda = div.head.string
		sutta_counter = 1
		if kanda == "verañjakaṇḍaṃ":
			kanda_no = "0."
		else:
			kanda_no = re.sub("^ ", "", kanda)
			kanda_no = re.sub("(^\\d*\\.)(.+)", "\\1", kanda_no)

	paras = div.children

	for para in paras:

		# get sutta name and number
		if "title" in str(para):
			if not re.findall("vaggo$", para.string):
				sutta_name = para.string
				subtitle_count = 1
				subtitle = ""

				sutta_no = f"{sutta_counter}."
				sutta_no = re.sub("(^\\d*\\.)(.+)", "\\1", sutta_name)
				sutta_clean = re.sub("(^\\d*\\. )(.+)", "\\2", sutta_name)
				sutta_counter +=1

		if "subhead" in str(para):
			subtitle = para.string
			subtitle_no = f"{subtitle_count}."
			subtitle_count += 1

		text = para.get_text()
		results = re.findall(search_sentence, text)

		for result in results: 
			result = text_cleaner(result[1])
			if kanda != "":
				
				# quick link
				print(f"[{result_counter}]\n")
				print(f"VIN ", end="")
				print(f"{book_no}", end="")
				print(f"{kanda_no}", end="")
				print(f"{sutta_no}", end="")
				if subtitle != "":
					print(f"{subtitle_no}", end="")
				
				# name of the book
				print(f"\t{sutta_clean}", end="")
				if subtitle != "":
					print(f", {subtitle}", end="")

				print(f"\t{result}")

				print(f"({book} {kanda} {sutta_name} {subtitle})\n")

			result_counter += 1



2022-09-18 05:22:17 vin01m.mul.xml
